<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1236976d5d2b05c4c122c446f4f72d4f4854ddf8c359bc945f1a53c27ae20ec0
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 09:16:32
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 21.43 K (0.15%)
Current PnL: -16.54 L (-11.12%)
CY Booked + Current PnL: -5.10 L (-3.43%)
-------------------
Total profit:  2.15 L
Total loss:  -18.69 L
-------------------
Total Booked + Current PnL: 21.88 L (17.98%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.0%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.05 L (56.66%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 10.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.46,64.0,H-MC,3.48,195720.0,31050.0,12272.0,-0.19,18.86,6.27,26.31,89.0,2.53,1.43,28.34,X40N,NTT,AC
76,TTKPRESTIG,770.00,105.34,59.0,M-SC,3.68,88464.0,-12313.0,12403.0,0.25,-12.22,14.02,0.09,245.0,-0.99,0.65,16.22,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,2.42,62.0,H-MC,2.71,214740.0,22998.0,14753.0,0.38,11.99,6.87,19.69,99.0,1.56,1.57,19.61,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.94,112010.0,11506.0,17597.0,0.02,11.45,15.71,28.96,79.0,0.65,0.82,49.30,MH,ATH,METALS
56,RELIANCE,1533.00,-11.79,60.0,H-LC,4.96,220740.0,10374.0,18410.0,0.00,4.93,8.34,13.68,37.0,0.56,1.61,22.27,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.46,57.0,H-SC,16.92,201799.0,1845.0,95390.0,0.07,0.92,47.27,48.63,115.0,0.02,1.47,12.58,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.31,48.0,H-MC,2.17,137114.0,1326.0,41422.0,0.28,0.98,30.21,31.48,107.0,0.03,1.00,13.21,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.77,61.0,H-MC,1.93,249865.0,2134.0,72936.0,0.46,0.86,29.19,30.30,92.0,0.03,1.83,14.27,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.81,52.0,H-MC,4.09,200250.0,-570.0,69347.0,-0.02,-0.28,34.63,34.25,80.0,-0.01,1.46,4.90,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-16.43,41.0,H-LC,6.59,300721.0,-15121.0,127776.0,-0.46,-4.79,42.49,35.67,5.0,-0.12,2.20,8.40,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.45,45.0,H-LC,7.75,218504.0,-33264.0,86484.0,0.17,-13.21,39.58,21.13,27.0,-0.38,1.60,17.53,X40,ATH,PAINTS
41,ITC,452.00,-37.50,52.0,H-LC,2.36,201321.0,1183.0,19709.0,0.04,0.59,9.79,10.44,4.0,0.06,1.47,6.19,X40,NTT,FMCG
79,UNITDSPR,1644.00,-12.66,53.0,H-LC,7.76,228794.0,-5350.0,53973.0,0.21,-2.28,23.59,20.77,86.0,-0.10,1.67,4.48,X40N,NTT,BREWERIES
43,JIOFIN,387.00,0.71,54.0,H-LC,13.15,273652.0,6287.0,59164.0,0.27,2.35,21.62,24.48,48.0,0.11,2.00,58.62,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.29,70.0,H-LC,1.48,248512.0,2361.0,76840.0,-0.04,0.96,30.92,32.18,16.0,0.03,1.82,37.35,X200,ATH,IT
41,ITC,452.0,-37.50,52.0,H-LC,2.36,201321.0,1183.0,19709.0,0.04,0.59,9.79,10.44,4.0,0.06,1.47,6.19,X40,NTT,FMCG
50,NESTLEIND,1377.0,-8.35,57.0,H-LC,4.32,282204.0,16778.0,40017.0,-0.26,6.32,14.18,21.40,11.0,0.42,2.06,13.86,XY25,NTT,FMCG
56,RELIANCE,1533.0,-11.79,60.0,H-LC,4.96,220740.0,10374.0,18410.0,0.00,4.93,8.34,13.68,37.0,0.56,1.61,22.27,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-18.75,58.0,H-LC,5.71,216830.0,12330.0,29077.0,0.29,6.03,13.41,20.25,10.0,0.42,1.58,15.29,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-64.43,58.0,H-MC,4.01,253362.0,-48389.0,209809.0,2.20,-16.04,82.81,53.50,116.0,-0.23,1.85,12.53,XY24,NTT,FMCG
6,ASIANTILES,137.00,7100.00,68.0,L-SC,5.23,79006.0,-14804.0,91418.0,1.86,-15.78,115.71,81.67,269.0,-0.16,0.58,53.00,XR,NTT,CERAMICS
63,SHALBY,327.00,1118.50,70.0,M-SC,4.64,165627.0,-15516.0,60984.0,1.56,-8.57,36.82,25.10,235.0,-0.25,1.21,32.73,XY24,NTT,HEALTHCARE
2,ACC,3906.00,-36.77,63.0,M-MC,4.84,137240.0,-51031.0,147904.0,1.18,-27.10,107.77,51.45,174.0,-0.35,1.00,5.46,XY24,BTT,CEMENT
70,TANLA,1963.11,-19.51,70.0,H-SC,12.12,216342.0,-54419.0,384375.0,1.06,-20.10,177.67,121.86,133.0,-0.14,1.58,69.24,AR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-326.45,54.0,H-SC,6.28,125800.0,-37805.0,159124.0,-1.21,-23.11,126.49,74.15,139.0,-0.24,0.92,61.26,XR,NTT,CHEMICALS
80,VAIBHAVGBL,521.00,64.74,69.0,H-SC,6.60,142675.0,-40100.0,151164.0,-0.68,-21.94,105.95,60.77,125.0,-0.27,1.04,27.79,XR,NTT,JEWELLERY
53,RAJESHEXPO,518.00,1878.54,65.0,L-SC,2.61,53980.0,-83197.0,83291.0,-0.62,-60.65,154.30,0.07,267.0,-1.00,0.39,34.25,OX40N,NTT,JEWELLERY
58,ROUTE,2236.84,-46.55,48.0,H-SC,17.47,87890.0,-45176.0,138031.0,-0.59,-33.95,157.05,69.78,118.0,-0.33,0.64,2.62,SR,ATH,IT
55,RELAXO,1176.00,-32.04,56.0,H-SC,1.96,92599.0,-52561.0,128490.0,-0.58,-36.21,138.76,52.31,136.0,-0.41,0.68,23.95,X40N,NTT,FOOTWEAR


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,105.34,59.0,M-SC,3.68,88464.0,-12313.0,12403.0,0.25,-12.22,14.02,0.09,245.0,-0.99,0.65,16.22,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,53.0,H-SC,9.41,122796.0,-19683.0,39209.0,-0.07,-13.81,31.93,13.70,163.0,-0.50,0.90,44.93,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.87,66.0,H-SC,1.24,228960.0,-40707.0,77045.0,0.45,-15.10,33.65,13.47,138.0,-0.53,1.67,15.80,XY24,NTT,PAINTS
18,CERA,9475.0,-19.57,49.0,H-SC,2.08,146430.0,-29473.0,71502.0,0.30,-16.76,48.83,23.89,149.0,-0.41,1.07,26.46,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.51,54.0,H-MC,7.43,108927.0,-22908.0,65127.0,-0.01,-17.38,59.79,32.03,98.0,-0.35,0.80,22.01,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,59.0,H-LC,12.29,298149.0,-47807.0,122658.0,-0.15,-13.82,41.14,21.64,1.0,-0.39,2.18,5.62,X40,ATH,IT
39,INFY,2275.00,-15.29,59.0,H-LC,8.12,329788.0,16900.0,154802.0,0.50,5.40,46.94,54.87,3.0,0.11,2.41,12.41,X40,BTT,IT
41,ITC,452.00,-37.50,52.0,H-LC,2.36,201321.0,1183.0,19709.0,0.04,0.59,9.79,10.44,4.0,0.06,1.47,6.19,X40,NTT,FMCG
82,VBL,671.64,-16.43,41.0,H-LC,6.59,300721.0,-15121.0,127776.0,-0.46,-4.79,42.49,35.67,5.0,-0.12,2.20,8.40,X40N,ATH,FMCG
1,ABB,7934.00,-37.43,69.0,H-LC,9.84,261614.0,-5.0,119217.0,0.26,-0.00,45.57,45.57,7.0,-0.00,1.91,11.32,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,45.15,65.0,M-MC,7.44,229129.0,4167.0,163484.0,-0.09,1.85,71.35,74.53,192.0,0.03,1.67,34.70,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.46,64.0,H-MC,3.48,195720.0,31050.0,12272.0,-0.19,18.86,6.27,26.31,89.0,2.53,1.43,28.34,X40N,NTT,AC
83,VOLTAS,1530.00,2.42,62.0,H-MC,2.71,214740.0,22998.0,14753.0,0.38,11.99,6.87,19.69,99.0,1.56,1.57,19.61,XY25,NTT,AC
29,HAVELLS,2069.16,0.77,61.0,H-MC,1.93,249865.0,2134.0,72936.0,0.46,0.86,29.19,30.30,92.0,0.03,1.83,14.27,X40,ATH,ELECTRICAL
17,CAMS,5211.76,0.38,67.0,H-SC,3.47,114187.0,12183.0,31744.0,0.07,11.94,27.80,43.06,122.0,0.38,0.83,31.10,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.94,112010.0,11506.0,17597.0,0.02,11.45,15.71,28.96,79.0,0.65,0.82,49.30,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.92,49.0,M-SC,0.64,99802.0,8304.0,70450.0,-0.35,9.08,70.59,86.07,223.0,0.12,0.73,44.93,XR,NTT,DURABLES
36,INDIAMART,4850.92,-52.21,40.0,H-SC,11.52,133369.0,10033.0,123726.0,-0.26,8.13,92.77,108.45,119.0,0.08,0.97,32.31,AR,ATH,MISC
85,WIPRO,420.00,-9.26,60.0,M-LC,5.65,159402.0,8457.0,100997.0,0.07,5.60,63.36,72.51,53.0,0.08,1.16,11.96,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,64.0,H-MC,6.28,183406.0,3598.0,108815.0,0.20,2.00,59.33,62.52,88.0,0.03,1.34,36.77,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7100.00,68.0,L-SC,5.23,79006.0,-14804.0,91418.0,1.86,-15.78,115.71,81.67,269.0,-0.16,0.58,53.00,XR,NTT,CERAMICS
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,3.21,83112.0,-19098.0,112492.0,-0.48,-18.69,135.35,91.37,208.0,-0.17,0.61,34.92,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.22,84403.0,-29146.0,69194.0,-0.00,-25.67,81.98,35.27,268.0,-0.42,0.62,106.58,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.50,89152.0,-11811.0,106341.0,-0.10,-11.70,119.28,93.63,148.0,-0.11,0.65,32.79,SR,ATH,CHEMICALS
48,MASFIN,398.61,-17.34,50.0,H-SC,12.98,94050.0,-3930.0,25535.0,-0.19,-4.01,27.15,22.05,152.0,-0.15,0.69,36.71,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.0,1118.50,70.0,M-SC,4.64,165627.0,-15516.0,60984.0,1.56,-8.57,36.82,25.10,235.0,-0.25,1.21,32.73,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.0,3900.00,64.0,M-SC,5.33,124714.0,-22654.0,74467.0,0.06,-15.37,59.71,35.16,236.0,-0.30,0.91,27.38,XY24,NTT,AUTO
74,TITAGARH,1548.0,1.18,68.0,H-SC,3.16,165725.0,-26290.0,105169.0,0.05,-13.69,63.46,41.08,158.0,-0.25,1.21,40.43,XY24,NTT,ENGINEERING
80,VAIBHAVGBL,521.0,64.74,69.0,H-SC,6.60,142675.0,-40100.0,151164.0,-0.68,-21.94,105.95,60.77,125.0,-0.27,1.04,27.79,XR,NTT,JEWELLERY
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,3.21,83112.0,-19098.0,112492.0,-0.48,-18.69,135.35,91.37,208.0,-0.17,0.61,34.92,XY25,NTT,FINANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.14
1,25,43.98
2,50,74.49


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.51
LC    32.13
MC    23.35
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.33
X40      15.44
X40N     11.93
XR       11.78
AR        8.76
XY25      8.26
OX40N     8.23
X200      1.82
MH        1.70
X5K       1.45
SR        1.29
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.25
H-LC    25.44
H-MC    20.37
M-SC    13.67
M-LC     5.63
M-MC     2.67
L-SC     1.59
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.69,-12.20,73.38
FMCG,11.93,-3.64,39.91
FINANCE,9.72,-15.56,60.82
MISC,7.43,-12.64,75.26
BANKS,6.33,-15.31,75.98
PAINTS,5.89,-14.57,31.65
ELECTRICAL,5.78,-4.13,61.53
AC,3.72,4.92,11.00
INSURANCE,3.44,-6.73,43.33


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2827119.0,22
XR,1298584.0,14
AR,1296977.0,9
X40,736689.0,10
X40N,570439.0,9
OX40N,520603.0,10
XY25,333217.0,6
SR,244372.0,2
X5K,119467.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3403790.0,29
M-SC,1375169.0,17
H-LC,1211150.0,14
H-MC,1081973.0,15
M-LC,378838.0,4
M-MC,311388.0,2
L-SC,243903.0,3
L-MC,60387.0,1
L-LC,38863.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1023020.0      6
M-SC       XY24       793780.0      7
H-SC       AR         785376.0      5
           XR         752416.0      7
H-MC       XY24       481393.0      4
H-LC       X40        454535.0      5
           AR         351358.0      2
M-MC       XY24       311388.0      2
H-SC       X40N       285658.0      4
           OX40N      249391.0      4
           SR         244372.0      2
H-MC       X40        211651.0      4
H-LC       X40N       210826.0      3
L-SC       XR         160612.0      2
M-SC       AR         160243.0      2
M-LC       XY24       158374.0      2
H-MC       XY25       123435.0      2
M-SC       OX40N      122794.0      4
M-LC       X5K        119467.0      1
M-SC       XR         115357.0      2
           XY25       112492.0      1
H-MC       XR         108815.0      1
M-LC       XR         100997.0      1
L-SC       OX40N       83291.0      1
H-LC       X200        76840.0      1
H-MC       X40N        73955.0      2
M-SC       X40         70503.0      1
H-MC       OX40N       65127.0      1
H-SC       MH          63557.0      1
L-MC       XR          60387.0      1
H-LC       XY24        59164.0      1
           XY25        58427.0      2
L-LC       XY25        38863.0      1
H-MC       MH          17597.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
